# TSLA Intraday ATR Analysis

This notebook calculates the 20-day moving median of the Average True Range (ATR) for TSLA using intraday minute-level data.

## Methodology
- **Intraday ATR**: For each trading day, we calculate the range as daily high - daily low
- **20-Day Moving Median**: We then apply a 20-day rolling median to smooth the daily ATR values
- **Data Source**: TSLA minute-level data for August 2025

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 50)

print("Libraries imported successfully!")

## 1. Load and Prepare Data

In [ ]:
# Load the TSLA minute data
df = pd.read_csv('tsla_minute_data_august_2025.csv')

# Convert time column to datetime
df['time'] = pd.to_datetime(df['time'])

# Extract date for grouping
df['date'] = df['time'].dt.date

print(f"Data loaded successfully!")
print(f"Total records: {len(df):,}")
print(f"Date range: {df['time'].min()} to {df['time'].max()}")
print(f"\nFirst few rows:")
df.head()

## 2. Calculate Daily High, Low, Open and Close Prices

In [ ]:
# Group by date and get the daily OHLC values
daily_prices = df.groupby('date').agg({
    'open': 'first',   # First minute's open price of the day
    'close': 'last',   # Last minute's close price of the day
    'high': 'max',     # Highest price of the day
    'low': 'min',      # Lowest price of the day
    'time': ['first', 'last']  # First and last timestamps
}).reset_index()

# Flatten column names
daily_prices.columns = ['date', 'daily_open', 'daily_close', 'daily_high', 'daily_low', 'first_time', 'last_time']

print(f"Daily prices calculated for {len(daily_prices)} trading days")
print("\nDaily price summary:")
daily_prices[['date', 'daily_open', 'daily_close', 'daily_high', 'daily_low']].head(10)

## 3. Calculate Daily ATR (High - Low)

In [ ]:
# Calculate the intraday ATR (high - low)
daily_prices['intraday_atr'] = daily_prices['daily_high'] - daily_prices['daily_low']

# Also calculate the open-close range for comparison
daily_prices['open_close_range'] = abs(daily_prices['daily_close'] - daily_prices['daily_open'])

# Calculate percentage move
daily_prices['pct_change'] = ((daily_prices['daily_close'] - daily_prices['daily_open']) / daily_prices['daily_open']) * 100

# Calculate ATR as percentage of closing price
daily_prices['atr_pct'] = (daily_prices['intraday_atr'] / daily_prices['daily_close']) * 100

print("Daily ATR Statistics (High - Low):")
print(f"Mean Intraday ATR: ${daily_prices['intraday_atr'].mean():.2f}")
print(f"Max Intraday ATR: ${daily_prices['intraday_atr'].max():.2f}")
print(f"Min Intraday ATR: ${daily_prices['intraday_atr'].min():.2f}")
print(f"Std Dev: ${daily_prices['intraday_atr'].std():.2f}")

print("\nDaily ATR values:")
daily_prices[['date', 'daily_high', 'daily_low', 'intraday_atr', 'atr_pct']].round(2)

## 4. Calculate 20-Day Moving Median of ATR

In [ ]:
# Calculate the 20-day moving median of the intraday ATR
daily_prices['atr_median20'] = daily_prices['intraday_atr'].rolling(window=20, min_periods=1).median()

# Also calculate other moving medians for context
daily_prices['atr_median5'] = daily_prices['intraday_atr'].rolling(window=5, min_periods=1).median()
daily_prices['atr_median10'] = daily_prices['intraday_atr'].rolling(window=10, min_periods=1).median()

# For comparison, also calculate moving average
daily_prices['atr_ma20'] = daily_prices['intraday_atr'].rolling(window=20, min_periods=1).mean()

print("20-Day Moving Median ATR calculated!")
print(f"\nFinal 20-Day Median ATR: ${daily_prices['atr_median20'].iloc[-1]:.2f}")
print(f"Average 20-Day Median ATR: ${daily_prices['atr_median20'].mean():.2f}")
print(f"\nComparison with Moving Average:")
print(f"Final 20-Day MA ATR: ${daily_prices['atr_ma20'].iloc[-1]:.2f}")

# Display the last few rows with all calculations
print("\nLast 10 days with moving medians:")
daily_prices[['date', 'intraday_atr', 'atr_median5', 'atr_median10', 'atr_median20']].tail(10).round(2)

## 5. Visualization with Matplotlib

In [ ]:
# Create the main plot
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Daily ATR with 20-day moving median
ax1.plot(daily_prices['date'], daily_prices['intraday_atr'], 
         label='Daily ATR (High-Low)', color='steelblue', linewidth=1, alpha=0.7)
ax1.plot(daily_prices['date'], daily_prices['atr_median20'], 
         label='20-Day Median', color='red', linewidth=2)
ax1.plot(daily_prices['date'], daily_prices['atr_ma20'], 
         label='20-Day Mean', color='orange', linewidth=1, linestyle='--', alpha=0.7)
ax1.fill_between(daily_prices['date'], 0, daily_prices['intraday_atr'], 
                  alpha=0.3, color='steelblue')

ax1.set_title('TSLA Intraday ATR (High-Low) with 20-Day Moving Median', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('ATR ($)', fontsize=12)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Add horizontal line for median ATR
median_atr = daily_prices['intraday_atr'].median()
ax1.axhline(y=median_atr, color='green', linestyle='--', alpha=0.5, 
            label=f'Median ATR: ${median_atr:.2f}')

# Plot 2: Multiple moving medians comparison
ax2.plot(daily_prices['date'], daily_prices['intraday_atr'], 
         label='Daily ATR', color='gray', linewidth=0.5, alpha=0.5)
ax2.plot(daily_prices['date'], daily_prices['atr_median5'], 
         label='5-Day Median', color='orange', linewidth=1.5)
ax2.plot(daily_prices['date'], daily_prices['atr_median10'], 
         label='10-Day Median', color='blue', linewidth=1.5)
ax2.plot(daily_prices['date'], daily_prices['atr_median20'], 
         label='20-Day Median', color='red', linewidth=2)

ax2.set_title('ATR Moving Medians Comparison', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('ATR ($)', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Plot 3: ATR as percentage of price
ax3.bar(daily_prices['date'], daily_prices['atr_pct'], color='purple', alpha=0.6)
ax3.axhline(y=daily_prices['atr_pct'].mean(), color='red', linestyle='--', 
            label=f"Mean: {daily_prices['atr_pct'].mean():.2f}%")

ax3.set_title('Daily ATR as Percentage of Closing Price', fontsize=14, fontweight='bold')
ax3.set_xlabel('Date', fontsize=12)
ax3.set_ylabel('ATR %', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Visualization complete!")

## 6. Statistical Analysis

In [ ]:
# Create a summary statistics table
summary_stats = pd.DataFrame({
    'Metric': ['Mean', 'Median', 'Std Dev', 'Min', 'Max', '25th Percentile', '75th Percentile'],
    'Daily ATR (High-Low)': [
        daily_prices['intraday_atr'].mean(),
        daily_prices['intraday_atr'].median(),
        daily_prices['intraday_atr'].std(),
        daily_prices['intraday_atr'].min(),
        daily_prices['intraday_atr'].max(),
        daily_prices['intraday_atr'].quantile(0.25),
        daily_prices['intraday_atr'].quantile(0.75)
    ],
    '20-Day Median ATR': [
        daily_prices['atr_median20'].mean(),
        daily_prices['atr_median20'].median(),
        daily_prices['atr_median20'].std(),
        daily_prices['atr_median20'].min(),
        daily_prices['atr_median20'].max(),
        daily_prices['atr_median20'].quantile(0.25),
        daily_prices['atr_median20'].quantile(0.75)
    ]
})

summary_stats = summary_stats.round(2)
print("Summary Statistics for TSLA Intraday ATR:")
print(summary_stats.to_string(index=False))

# Calculate volatility metrics
print("\n" + "="*50)
print("Volatility Analysis:")
print(f"ATR as % of average price: {(daily_prices['intraday_atr'].mean() / daily_prices['daily_close'].mean()) * 100:.2f}%")
print(f"Coefficient of Variation (ATR): {(daily_prices['intraday_atr'].std() / daily_prices['intraday_atr'].mean()):.2f}")
print(f"Days with ATR > $15: {len(daily_prices[daily_prices['intraday_atr'] > 15])} ({len(daily_prices[daily_prices['intraday_atr'] > 15])/len(daily_prices)*100:.1f}%)")
print(f"Days with ATR > $20: {len(daily_prices[daily_prices['intraday_atr'] > 20])} ({len(daily_prices[daily_prices['intraday_atr'] > 20])/len(daily_prices)*100:.1f}%)")

# Compare ATR with open-close range
print("\n" + "="*50)
print("ATR vs Open-Close Range Comparison:")
print(f"Mean ATR (High-Low): ${daily_prices['intraday_atr'].mean():.2f}")
print(f"Mean Open-Close Range: ${daily_prices['open_close_range'].mean():.2f}")
print(f"Ratio (ATR/Open-Close): {daily_prices['intraday_atr'].mean() / daily_prices['open_close_range'].mean():.2f}x")

## 7. Export Results

In [ ]:
# Save the calculated ATR data to a CSV file
output_df = daily_prices[['date', 'daily_high', 'daily_low', 'intraday_atr', 'atr_median5', 'atr_median10', 'atr_median20', 'atr_pct']]
output_df.to_csv('tsla_atr_analysis_results.csv', index=False)

print("Results saved to 'tsla_atr_analysis_results.csv'")
print("\nFinal Results Preview:")
output_df.tail(10).round(2)

## Conclusions

### Key Findings:
1. **Average Intraday ATR**: The average daily range (high - low) shows the true volatility
2. **20-Day Moving Median**: Provides a robust smoothed view of volatility trends (less affected by outliers than mean)
3. **Volatility Patterns**: The data shows how TSLA's intraday volatility changes over time

### Interpretation:
- ATR (High-Low) represents the true trading range for each day
- The 20-day moving median is more stable than the mean, less influenced by extreme values
- This metric can be useful for:
  - Risk management and position sizing
  - Setting stop-loss levels based on actual price ranges
  - Identifying volatility expansion/contraction periods
  - Options trading strategies